In [66]:
#Equipo:  Jesus Alejandro Valencia Valadez
#         Donovan Gared Gracia Orozco

In [ ]:
%%capture 
!pip install rpy2==3.5.1

In [67]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [68]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [69]:
%%R
library(readr)
library(magrittr)
library(dplyr)

In [70]:
%%R
data <- read.csv("drive/MyDrive/ds_salaries.csv")

In [71]:
%%R
colnames(data)

 [1] "work_year"          "experience_level"   "employment_type"   
 [4] "job_title"          "salary"             "salary_currency"   
 [7] "salary_in_usd"      "employee_residence" "remote_ratio"      
[10] "company_location"   "company_size"      


1. LA columna "salay_in_usd"muestra el salario anual en usd de cada puesto de trabajo. 
Si un dolar equivale a 17.81 mxn, elabore una columna que muestre la conversion a salario
 mensual en pesos y muestre el top 10 de los titulos profesionales que tienen el mejor sueldo en ciencia de datos

In [72]:
%%R
data %>% 
group_by(salary_in_usd) %>%
mutate(conversion_pesos = (salary_in_usd * 17.81)/12) %>%
select(job_title,salary_in_usd, conversion_pesos)%>%
arrange(desc(conversion_pesos))%>%
head(n=10)

# A tibble: 10 × 3
# Groups:   salary_in_usd [10]
   job_title                          salary_in_usd conversion_pesos
   <chr>                                      <int>            <dbl>
 1 Research Scientist                        450000          667875 
 2 Data Analyst                              430967          639627.
 3 AI Scientist                              423834          629040.
 4 Applied Machine Learning Scientist        423000          627802.
 5 Principal Data Scientist                  416000          617413.
 6 Data Scientist                            412000          611477.
 7 Data Analytics Lead                       405000          601087.
 8 Data Analyst                              385000          571404.
 9 Applied Data Scientist                    380000          563983.
10 Data Architect                            376080          558165.


2. Suponga que desea aplicar este año para un puesto de los presentados en este conjunto de datos. Si su nivel de experencia es EN y quiere obtener el sueldo más alto posible, ¿Qué título le conviene más obtener? Obtenga el salario promedio de cada puesto cuyo año sea el año actual para encintrar la respuesta y muestrela junto con su salario mensual en mxn

In [73]:
%%R
data %>% 
group_by(experience_level) %>%
filter(experience_level == "EN")%>%
filter(work_year == "2023")%>%
mutate(conversion_pesos = (salary_in_usd * 17.81)/12) %>%
select(work_year,job_title, conversion_pesos)%>%
arrange(desc(conversion_pesos))%>%
head(1)

Adding missing grouping variables: `experience_level`
# A tibble: 1 × 4
# Groups:   experience_level [1]
  experience_level work_year job_title          conversion_pesos
  <chr>                <int> <chr>                         <dbl>
1 EN                    2023 Research Scientist          326517.


3. Existen registros de trabajadores que radican en México. Averigue qué título profesional, nivel de expenencia y salario mensual en mxn tienen y ordene los registros de mayor a menor por su salario.

In [74]:
%%R
data %>%
filter(grepl("MX", company_location)) %>%
mutate(conversion_pesos = salary_in_usd * 17.81 / 12) %>%
select(job_title, experience_level, conversion_pesos) %>%
arrange(desc(conversion_pesos)) %>%
head(n=10)

                   job_title experience_level conversion_pesos
1  Machine Learning Engineer               SE        378462.50
2  Machine Learning Engineer               SE        274570.83
3       Data Science Manager               SE        222625.00
4       Data Science Manager               SE        133575.00
5              Data Engineer               MI        106860.00
6              Data Engineer               MI         89050.00
7      Data Science Engineer               SE         89050.00
8  Machine Learning Engineer               MI         53430.00
9              Data Engineer               SE         49735.91
10            Data Scientist               MI         44525.00


4. Para alquien con experiencia "EN" que desea buscar un puesto de tiempo completo y ganar lo más posible, ¿le conviene trabajar de 
manera remota o presencial?

In [75]:
%%R
data %>% 
filter(grepl("EN", experience_level)) %>%
filter(employment_type == "FT") %>%
group_by(remote_ratio) %>%
summarize(mean_salary_in_usd = mean(salary_in_usd))

#Le conviene trabajar de manera presencial

# A tibble: 3 × 2
  remote_ratio mean_salary_in_usd
         <int>              <dbl>
1            0            104582.
2           50             65344 
3          100             67016.


5. Cantidad de registros donde el país de residencia de la compañia y del trabajador son diferentes

In [76]:
%%R
data %>% 
group_by(employee_residence) %>%
filter(company_location != employee_residence)%>%
nrow()

[1] 96


6. Salario mensual promedio en pesos mexicanos para un trabajador con experiencia "EN° en una empresa de tamaño S, L y M de acuerdo con los datos del año *2022*

In [77]:
%%R
data %>% 
filter(experience_level == "EN" & work_year == "2022") %>%
mutate(Salario_mensual_mx = (salary_in_usd * 17.81)/12) %>%
group_by(company_size) %>%
summarise(Prom_Salario = mean(Salario_mensual_mx))

# A tibble: 3 × 2
  company_size Prom_Salario
  <chr>               <dbl>
1 L                 105853.
2 M                 127823.
3 S                  83360.


7. Top 10 Cantidad de trabajadores de cada pais registrado que trabajan en una compañía ubicada en Estados Unidos (ordenados de mayor a menor cantidad)

In [78]:
%%R
data %>% 
filter(company_location == "US") %>%
group_by(employee_residence) %>%
summarize(Pais = n()) %>% 
arrange(desc(Pais)) %>%
head(n=10)

# A tibble: 10 × 2
   employee_residence  Pais
   <chr>              <int>
 1 US                  2999
 2 IN                     7
 3 BR                     3
 4 ES                     3
 5 AR                     2
 6 BO                     2
 7 PT                     2
 8 RU                     2
 9 BE                     1
10 BG                     1


8. ¿Cuál es la equivalencia en promedio de cada moneda utilizada para la conversión a dólares?

In [82]:
%%R
data %>% 
group_by(salary_currency) %>% 
summarize(promedio =mean(salary), promedio_usd = mean(salary_in_usd)) %>% 
summarize(cambio= salary_currency, promtot = promedio/promedio_usd)

# A tibble: 20 × 2
   cambio promtot
   <chr>    <dbl>
 1 AUD      1.43 
 2 BRL      5.27 
 3 CAD      1.29 
 4 CHF      0.946
 5 CLP    759.   
 6 CZK     23.4  
 7 DKK      6.41 
 8 EUR      0.918
 9 GBP      0.809
10 HKD      7.84 
11 HUF    319.   
12 ILS      3.54 
13 INR     77.0  
14 JPY    109.   
15 MXN     21.5  
16 PLN      4.19 
17 SGD      1.35 
18 THB     34.0  
19 TRY      8.92 
20 USD      1    
